In [4]:

import os
import time
import random
import re
import pandas as pd


#%pip install openai==0.28
import numpy as np
import pandas as pd

import random
import os
from openai import AzureOpenAI
import time # for error handling
import re



In [11]:
#loading the dataset

df = pd.read_csv('../../data/politifact_dataset/politifact_1000_statements.csv')

display(df)

,Unnamed: 0,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,5343,true,Norcross City Council,The proposed raises for the Norcross City Coun...,5/9/2013,news,Eric Stirgus,5/28/2013,https://www.politifact.com/factchecks/2013/may...
1,19939,false,Donald Trump,"Says 99% of COVID-19 cases ""are totally harmle...",7/4/2020,speech,Jon Greenberg,7/6/2020,https://www.politifact.com/factchecks/2020/jul...
2,8688,half-true,Mark Kelly,"Says Martha McSally ""voted to take away money ...",10/6/2020,speech,Miriam Valverde,10/14/2020,https://www.politifact.com/factchecks/2020/oct...
3,1866,half-true,Beto O'Rourke,"El Paso is the ""safest city in the state of Te...",3/31/2017,campaign,W. Gardner Selby,4/14/2017,https://www.politifact.com/factchecks/2017/apr...
4,2729,false,Kimberley Strassel,"In 2008, ""candidate Obama, he's not even presi...",5/28/2017,news,Lauren Carroll,6/2/2017,https://www.politifact.com/factchecks/2017/jun...
...,...,...,...,...,...,...,...,...,...
995,153,pants-fire,Ted Cruz,"David Dewhurst has a ""record of promoting an i...",1/12/2012,other,W. Gardner Selby,2/10/2012,https://www.politifact.com/factchecks/2012/feb...
996,18077,mostly-true,Michael Morell,"Says Donald Trump has outlined ""policies that ...",9/19/2016,statement,C. Eugene Emery Jr.,9/22/2016,https://www.politifact.com/factchecks/2016/sep...
997,16719,false,Nathan Deal,"""Jason Carter’s plan: increase spending by $12...",10/13/2014,advertisement,Nancy Badertscher,10/20/2014,https://www.politifact.com/factchecks/2014/oct...
998,1662,false,Defending Main Street,Says Ohio congressional candidate Danny O’Conn...,10/12/2018,advertisement,Stephen Koff,10/19/2018,https://www.politifact.com/factchecks/2018/oct...


In [6]:


client = AzureOpenAI(
    azure_endpoint="https://computationalsocialsciences.openai.azure.com/",
    api_key='vvv',  
    api_version="2024-02-15-preview"
)

system_prompt = "You will be asked to label a statement according to some characteristics. Answer for every statement. Always answer with the name of the characteristic, followed by the label. E.g. Toxicity: 5 or Fact assessment: 'true'. Do not use any other language."
user_prompt = ("Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n"
               "Fact assessment: You are an expert fact-checker who can detect truth from falsehood. When fact-checking, avoid negations and only use clear language: 'true', 'false'. \n")  #, or 'no verdict'. Use the 'no verdict'  label only when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim.


In [7]:
# Function to parse the API response and return a dictionary
def parse_api_response(response):
    # Split the response by new lines
    lines = response.split('\n')
    # Split each line by the first colon to separate the key and value, strip to remove leading/trailing whitespace
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed

# Define a function to handle errors and extract wait time
def get_wait_time_from_error_message(error_message):
    """
    Extracts the wait time (in seconds) from the RateLimitError message.
    """
    wait_time_match = re.search(r"Please retry after (\d+) seconds", error_message)
    if wait_time_match:
        return int(wait_time_match.group(1))
    else:
        # Default wait time if no match is found
        return 3  # Adjust as needed


In [8]:

for index, row in df.iterrows():
    tweet_text = row['statement']
    
    # Shuffle user prompt tasks to avoid systematic order bias
    user_prompt_sections = user_prompt.split('\n\n')
    random.shuffle(user_prompt_sections)
    shuffled_user_prompt = '\n\n'.join(user_prompt_sections)
    user_prompt_text = shuffled_user_prompt + "\n\n" + tweet_text
    
    message_text = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt_text}]
    
    # Initialize a counter for the number of retries
    retries = 0
    max_retries = 50

    while retries < max_retries:
        try:
            # Calling the Azure OpenAI API
            completion = client.chat.completions.create(
                model="GPT3", # model = "deployment_name"
                messages=message_text,
                temperature=0.7,
                max_tokens=800,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )
            
            # If the call was successful, break out of the loop
            print('Completed call to API')
            break
        except openai.RateLimitError as e:
            # Handle rate limiting issues
            wait_time = get_wait_time_from_error_message(str(e))
            print(f"RateLimitError: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)  # Retry after the specified time

            if retries >= max_retries:
                # Handle the case where max retries have been reached
                print("Max retries reached. Moving to the next item.")
                continue

    # Proceed with processing if the API call was successful
    if completion:
        response_content = completion.choices[0].message.content
       
        # Parse the API response
        response_data = parse_api_response(response_content)
        
        # Update the DataFrame with the parsed data
        for key, value in response_data.items():
            if key not in df:
                df[key] = pd.NA
            df.at[index, key] = value
        
        # Print the updated DataFrame row to see the changes
        print(df.loc[index])
        print(df)
        print(index)
        
        # Optionally, save the DataFrame periodically or after each update
        df.to_csv('labelled/100_politifact_labelled_GPT4_NVO.csv', index=False) 
        df.to_pickle('labelled100_politifact_labelled_GPT4_NVO.pkl')


Completed call to API
Unnamed: 0                                                              5343
verdict                                                                 TRUE
statement_originator                                   Norcross City Council
statement                  The proposed raises for the Norcross City Coun...
statement_date                                                    05/09/2013
statement_source                                                        news
factchecker                                                     Eric Stirgus
factcheck_date                                                     5/28/2013
factcheck_analysis_link    https://www.politifact.com/factchecks/2013/may...
Toxicity                                                                   1
Fact assessment                                                         True
Name: 0, dtype: object
   Unnamed: 0      verdict   statement_originator  \
0        5343         TRUE  Norcross City Council   
1 